In [1]:
from PIL import Image, ImageOps

# Open the image
img = Image.open('cw1.jpg')

# Invert the image colors
inverted_img = ImageOps.invert(img.convert('RGB'))

# Save the inverted image
inverted_img.save('inverted_image.jpg')


In [7]:
from svgpathtools import svg2paths, Path, Line, CubicBezier, wsvg
import numpy as np
from scipy.interpolate import splprep, splev
import math

# ---------- CONFIG ----------
ANGLE_THRESHOLD = 150  # degrees; anything sharper is preserved
SMOOTHNESS = 0.5       # spline smoothing factor
SAMPLES_PER_SEGMENT = 10  # more = smoother
# ---------------------------

def angle_between(p1, p2, p3):
    v1 = np.array([p1.real - p2.real, p1.imag - p2.imag])
    v2 = np.array([p3.real - p2.real, p3.imag - p2.imag])
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 < 1e-6 or norm2 < 1e-6:
        return 180
    dot = np.dot(v1, v2) / (norm1 * norm2)
    angle = math.degrees(math.acos(np.clip(dot, -1.0, 1.0)))
    return angle

def dedup_points(points):
    result = [points[0]]
    for pt in points[1:]:
        if abs(pt - result[-1]) > 1e-6:
            result.append(pt)
    return result

# Load SVG path
paths, _ = svg2paths('Picture5.svg')  # adjust your filename
path = paths[0]

# Sample all segments into points
all_points = []
for segment in path:
    try:
        all_points.extend([segment.point(t / SAMPLES_PER_SEGMENT) for t in range(SAMPLES_PER_SEGMENT + 1)])
    except Exception:
        continue

# Deduplicate full list (for safety)
all_points = dedup_points(all_points)

# Detect corners by angle
sharp_points = [0]
for i in range(1, len(all_points) - 1):
    a = angle_between(all_points[i - 1], all_points[i], all_points[i + 1])
    if a < ANGLE_THRESHOLD:
        sharp_points.append(i)
sharp_points.append(len(all_points) - 1)

# Segment-wise smoothing
segments = []
for i in range(len(sharp_points) - 1):
    i0, i1 = sharp_points[i], sharp_points[i + 1]
    chunk = dedup_points(all_points[i0:i1 + 1])
    if len(chunk) < 4:
        # too small to smooth, use straight lines
        for j in range(len(chunk) - 1):
            segments.append(Line(chunk[j], chunk[j + 1]))
    else:
        try:
            x = np.array([p.real for p in chunk])
            y = np.array([p.imag for p in chunk])
            tck, _ = splprep([x, y], s=SMOOTHNESS)
            x_s, y_s = splev(np.linspace(0, 1, len(chunk) * 3), tck)
            for k in range(0, len(x_s) - 3, 3):
                p0 = complex(x_s[k], y_s[k])
                p1 = complex(x_s[k + 1], y_s[k + 1])
                p2 = complex(x_s[k + 2], y_s[k + 2])
                p3 = complex(x_s[k + 3], y_s[k + 3])
                segments.append(CubicBezier(p0, p1, p2, p3))
        except Exception as e:
            print(f"Skipping segment due to error: {e}")
            for j in range(len(chunk) - 1):
                segments.append(Line(chunk[j], chunk[j + 1]))

# Export result
new_path = Path(*segments)
wsvg([new_path], filename='smoothed_output.svg')

print("✅ Smoothing complete. Output saved to smoothed_output.svg")


✅ Smoothing complete. Output saved to smoothed_output.svg


In [8]:
from svgpathtools import svg2paths, Path, Line, CubicBezier, wsvg
import numpy as np
from scipy.interpolate import splprep, splev
import math

# ---------- CONFIG ----------
ANGLE_THRESHOLD = 150      # degrees; anything sharper is preserved
SMOOTHNESS = 0.5           # spline smoothing factor
SAMPLES_PER_SEGMENT = 10   # more = smoother
INPUT_FILE = 'Picture5.svg'   # your input file
OUTPUT_FILE = 'smoothed_output5.svg'
# ---------------------------

def angle_between(p1, p2, p3):
    v1 = np.array([p1.real - p2.real, p1.imag - p2.imag])
    v2 = np.array([p3.real - p2.real, p3.imag - p2.imag])
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 < 1e-6 or norm2 < 1e-6:
        return 180
    dot = np.dot(v1, v2) / (norm1 * norm2)
    angle = math.degrees(math.acos(np.clip(dot, -1.0, 1.0)))
    return angle

def dedup_points(points):
    result = [points[0]]
    for pt in points[1:]:
        if abs(pt - result[-1]) > 1e-6:
            result.append(pt)
    return result

def flip_y_point(pt, y_center):
    return complex(pt.real, 2 * y_center - pt.imag)

def flip_y_segment(segment, y_center):
    if isinstance(segment, CubicBezier):
        return CubicBezier(
            flip_y_point(segment.start, y_center),
            flip_y_point(segment.control1, y_center),
            flip_y_point(segment.control2, y_center),
            flip_y_point(segment.end, y_center)
        )
    elif isinstance(segment, Line):
        return Line(
            flip_y_point(segment.start, y_center),
            flip_y_point(segment.end, y_center)
        )
    else:
        return segment  # skip unsupported segment types

# Load SVG path
paths, _ = svg2paths(INPUT_FILE)
path = paths[0]

# Sample all points
all_points = []
for segment in path:
    try:
        all_points.extend([segment.point(t / SAMPLES_PER_SEGMENT) for t in range(SAMPLES_PER_SEGMENT + 1)])
    except Exception:
        continue

# Deduplicate points
all_points = dedup_points(all_points)

# Find Y bounds for flipping later
y_vals = [pt.imag for pt in all_points]
y_min, y_max = min(y_vals), max(y_vals)
y_center = (y_min + y_max) / 2

# Detect sharp points
sharp_points = [0]
for i in range(1, len(all_points) - 1):
    a = angle_between(all_points[i - 1], all_points[i], all_points[i + 1])
    if a < ANGLE_THRESHOLD:
        sharp_points.append(i)
sharp_points.append(len(all_points) - 1)

# Smooth between sharp points
segments = []
for i in range(len(sharp_points) - 1):
    i0, i1 = sharp_points[i], sharp_points[i + 1]
    chunk = dedup_points(all_points[i0:i1 + 1])
    if len(chunk) < 4:
        for j in range(len(chunk) - 1):
            segments.append(Line(chunk[j], chunk[j + 1]))
    else:
        try:
            x = np.array([p.real for p in chunk])
            y = np.array([p.imag for p in chunk])
            tck, _ = splprep([x, y], s=SMOOTHNESS)
            x_s, y_s = splev(np.linspace(0, 1, len(chunk) * 3), tck)
            for k in range(0, len(x_s) - 3, 3):
                p0 = complex(x_s[k], y_s[k])
                p1 = complex(x_s[k + 1], y_s[k + 1])
                p2 = complex(x_s[k + 2], y_s[k + 2])
                p3 = complex(x_s[k + 3], y_s[k + 3])
                segments.append(CubicBezier(p0, p1, p2, p3))
        except Exception as e:
            print(f"Skipping segment due to error: {e}")
            for j in range(len(chunk) - 1):
                segments.append(Line(chunk[j], chunk[j + 1]))

# Flip Y-axis
flipped_segments = [flip_y_segment(seg, y_center) for seg in segments]

# Export to new SVG
new_path = Path(*flipped_segments)
wsvg([new_path], filename=OUTPUT_FILE)

print(f"✅ Smoothing complete. Output saved to {OUTPUT_FILE}")


✅ Smoothing complete. Output saved to smoothed_output5.svg


In [10]:
from svgpathtools import svg2paths2, wsvg, Path
import xml.etree.ElementTree as ET

def remove_border_segments(input_file, output_file, margin=20):
    paths, attributes, svg_attributes = svg2paths2(input_file)

    full_path = paths[0]
    full_attr = attributes[0]

    # Clean units (e.g., '524px') before converting to float
    width = float(svg_attributes.get("width", "1000").replace("px", ""))
    height = float(svg_attributes.get("height", "1000").replace("px", ""))

    subpaths = full_path.continuous_subpaths()

    filtered_subpaths = []
    for sp in subpaths:
        xmin, xmax, ymin, ymax = sp.bbox()
        if (
            xmin > margin and xmax < width - margin and
            ymin > margin and ymax < height - margin
        ):
            filtered_subpaths.append(sp)

    cleaned_path = sum(filtered_subpaths, Path())

    wsvg([cleaned_path], attributes=[full_attr], svg_attributes=svg_attributes, filename=output_file)

# Example usage
remove_border_segments("smoothed_output5.svg", "cleaned_output.svg")



ValueError: not enough values to unpack (expected 4, got 0)